In [21]:
print('Good day!!!')

Good day!!!


In [141]:
import csv

# загрузили файл данных
with open("/home/yurist/Рабочий стол/LINUX/COD/PYTHON/MLLearn/checkins.dat", 'r') as fo:
    lines = fo.readlines()
fo.close()

# убрали все пробелы из строки и заменили разделитель ' | ' на ','
#linesCSV = [(' '.join(line.split())).replace('|', ',')  for line in lines]
linesCSV = [(''.join(line.split())).split('|')  for line in lines]

# убрали границу разделитель
linesCSV.pop(1)
#print(linesCSV[0])


with open('/home/yurist/Рабочий стол/LINUX/COD/PYTHON/MLLearn/checkins.csv', 'w', newline='') as fo:
    writer = csv.writer(fo)
    writer.writerows(linesCSV)
fo.close()


In [151]:
import pandas as pd



# создали DataFrame
df = pd.read_csv('/home/yurist/Рабочий стол/LINUX/COD/PYTHON/MLLearn/checkins.csv', header=0, sep=',')
df.head()

/home/yurist/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-2117:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-2117:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-2117:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-2117:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-2117:42:58


In [148]:
df.dropna(subset=['latitude', 'longitude'], inplace=True)
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-2117:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-2117:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-2117:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-2117:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-2117:38:18


In [150]:
df.shape

(396634, 6)

In [113]:
//?

SyntaxError: invalid syntax (<ipython-input-113-2266a8cc5790>, line 1)